<a href="https://colab.research.google.com/github/KOdin2/machine_learning/blob/main/SVM_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import io
import pandas as pd
import sklearn
import csv


#####
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline


SVM Code

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Brass_July_dataset.csv to Brass_July_dataset.csv
Saving Aluminium_July_dataset.csv to Aluminium_July_dataset.csv
Saving Zinc_July_dataset.csv to Zinc_July_dataset.csv
Saving SSteel_July_dataset.csv to SSteel_July_dataset.csv


In [ ]:
def graph_important_features(model, x_train_df):

  print("Feature importances via feature_importances_ for Random Forest")
  importance = np.abs(model.feature_importances_)
  feature_names = np.array(x_train_df.columns)
  f, ax = plt.subplots(figsize=(30,5))
  plt.bar(height=importance, x=feature_names, )
  plt.xticks(rotation='vertical', fontsize = 16)
  plt.yticks(fontsize = 16)
  plt.xlabel("Feature", fontsize = 16)
  plt.ylabel("Gini importance", fontsize = 16)
  plt.show()


Best measurements to use for classification are:

precision,
recall,
accuracy,
f1,
roc_auc


In [ ]:
def testing_combination_arrays(max_varible):

  varible_combinations = []

  b = 1
  for value in range(0, max_varible):
    varible_combinations.append(b) 
    b+=1

  return varible_combinations


def calculate_cross_validation_value(y_train):

  arr = y_train.to_numpy()
  bin_arr = np.bincount(arr)
  return bin_arr.min()

In [ ]:
#@title Grid search with a given x and y and combination of varible. Also number of cv fold cross validation

def grid_search_RF_function(x_train, y_train, Varible_list_1, number_of_cv_folds):

  best_model_details =[]
  #create mdoel
  Algorithm = "SVM"
  model =  SVC(random_state=0,)
  min_max = MinMaxScaler()
  
  #Create a pipline, which first will train on max and min and then transform the output
  pipe = Pipeline(steps = [
                               ("min_max" , min_max ),
                               
                               ("model" , model )
                               
  ])

  #Set up parameter grid
  param_grid = [
        {'model__kernel': ['linear', 'rbf', 'poly'], 'model__C': Varible_list_1, }
  ]

  #scoring_type can be used to choose what method to compare the models on
  scoring_type = 'f1'
 
  grid_search = GridSearchCV(estimator= pipe, param_grid= param_grid,  cv= number_of_cv_folds , scoring=scoring_type,  return_train_score=True, n_jobs=-1)

  grid_search.fit(X= x_train, y= y_train)

  #Aquire best parameter for model
  best_paramater = grid_search.best_params_
  print("[INFO] Best parameter: " + str(best_paramater))

  #find best score and round to 4dp
  best_accuracy = round(grid_search.best_score_,4)
  print("[INFO] Best score: " + str(best_accuracy))

  #Append results to 
  best_model_details.append([ Algorithm,
                        scoring_type,
                        best_accuracy,
                        best_paramater['model__kernel'],
                        best_paramater['model__C'],
                        str(number_of_cv_folds),
                      
  ])


  return best_model_details, best_paramater

In [ ]:
def best_model_test(x_train, y_train, number_of_cv_folds, best_paramater, best_results, x_train_df):

  model_check = []

  #create mdoel
  Algorithm = "SVM"
  model = SVC(random_state=0, kernel = best_paramater['model__kernel'], C = best_paramater['model__C'])  
  min_max = MinMaxScaler()
  
  pipe = Pipeline(steps = [
                               ("min_max" , min_max ),
                               
                               ("model" , model )
                               
  ])

  y_train_pred = cross_val_predict(pipe, x_train, y_train, cv=number_of_cv_folds)

  conf = confusion_matrix(y_train, y_train_pred)
  print(conf)
  f1        = round(f1_score(y_train, y_train_pred), 4)
  precision = round(precision_score(y_train, y_train_pred), 4)
  recall    = round(recall_score(y_train, y_train_pred), 4)

  print("F1 score\tPurity\t\tRecovery")
  print(str(f1) + "\t\t" + str(precision) + "\t\t" + str(recall) + "\n\n")
    
  model_check.append([f1, precision, recall, conf, len(x_train.columns),  str(number_of_cv_folds)])


  return model_check

In [ ]:
header = ["Algorithm", "Scoring type", "Score", "kernel",  "C", "Cross-validation setting"]
model_check_header = ["f1", "Purity", "Recovery", "Confusion matrix", "Number of inputs", "Cross-validation setting"]

In [ ]:
best_results = []
model_check = []
best_features = []
print("[INFO] CODE START")
for key in uploaded:
    best_model_list = []  
    best_features = []
    reduction_mode = False

    print("File uploaded: " +str(key))
    loaded_file = pd.read_csv(io.BytesIO(uploaded[key]))

    x_train_df = loaded_file.drop(['Label'], axis='columns')
    x_train = x_train_df.copy()
    y_train = loaded_file.Label

    #calcualte the largest possible cross validation
    number_of_cv_folds = StratifiedKFold(calculate_cross_validation_value(y_train))
    print("[INFO] Number of CV folds: " + str(number_of_cv_folds))

    #Calualte the all feature combinations and create list for vaible combinations
    C_var = testing_combination_arrays(100)
    print("[INFO] C combinations: " + str(C_var))


    print("[INFO] Finding best model parameters with x_train")
    best_model_details, best_paramater = grid_search_RF_function(x_train, y_train, C_var, number_of_cv_folds)
    best_model_list.append(best_model_details)

    model_check = best_model_test(x_train, y_train, number_of_cv_folds, best_paramater, best_results, x_train_df)
    
    with open(str(key) + '_SVM' +  '.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(best_model_list)
        writer.writerow(model_check_header)
        writer.writerows(model_check)
        #writer.writerow(best_features_header)
        #writer.writerows(best_features)

    files.download(str(key) + '_SVM' + '.csv') 


[INFO] CODE START
File uploaded: Brass_July_dataset.csv
[INFO] Number of CV folds: StratifiedKFold(n_splits=55, random_state=None, shuffle=False)
[INFO] C combinations: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
[INFO] Finding best model parameters with x_train
[INFO] Best parameter: {'model__C': 5, 'model__kernel': 'poly'}
[INFO] Best score: 0.7636
[[221  13]
 [ 10  45]]
F1 score	Purity		Recovery
0.7965		0.7759		0.8182




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File uploaded: Aluminium_July_dataset.csv
[INFO] Number of CV folds: StratifiedKFold(n_splits=84, random_state=None, shuffle=False)
[INFO] C combinations: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
[INFO] Finding best model parameters with x_train
[INFO] Best parameter: {'model__C': 29, 'model__kernel': 'rbf'}
[INFO] Best score: 0.7857
[[198   7]
 [ 16  68]]
F1 score	Purity		Recovery
0.8553		0.9067		0.8095




<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File uploaded: SSteel_July_dataset.csv
[INFO] Number of CV folds: StratifiedKFold(n_splits=50, random_state=None, shuffle=False)
[INFO] C combinations: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
[INFO] Finding best model parameters with x_train
[INFO] Best parameter: {'model__C': 2, 'model__kernel': 'poly'}
[INFO] Best score: 0.92
[[232   7]
 [  2  48]]
F1 score	Purity		Recovery
0.9143		0.8727		0.96




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>